In [68]:
import pandas as pd

In [69]:
df = pd.read_csv("../data/voix_sms/voix_sms_onnet.csv", sep=',')

In [70]:
df.head()

,Unnamed: 0,traffic_type,traffic_direction,msisdn,call_date,call_time,duree,ms_location
0,521,VOIX,SORTANT,5634d3960f5dbf326b2eb01e1cf66124,20210720,180146,182.0,608-01-09020-30101
1,737,VOIX,ENTRANT,a0c8f562a4abc89230b30a30409f69bc,20210720,72925,116.0,608-01-09040-25025
2,740,VOIX,ENTRANT,86b09e4ed3f30d94853e413451f1a69b,20210720,103649,39.0,608-01-09028-02017
3,660,VOIX,SORTANT,b58b587be09e5c1963bf4deba19597cc,20210720,213222,2.0,608-01-09000-36753
4,411,VOIX,ENTRANT,907d1610586c9337c2b19b00eca3bc79,20210720,230420,287.0,608-01-09000-03947


In [71]:
df.rename(columns={'duree':'duration', 'msisdn':'caller_id', 'traffic_type': 'txn_type', 'ms_location': 'caller_antenna'}, inplace=True)

In [72]:
df.head()

,Unnamed: 0,txn_type,traffic_direction,caller_id,call_date,call_time,duration,caller_antenna
0,521,VOIX,SORTANT,5634d3960f5dbf326b2eb01e1cf66124,20210720,180146,182.0,608-01-09020-30101
1,737,VOIX,ENTRANT,a0c8f562a4abc89230b30a30409f69bc,20210720,72925,116.0,608-01-09040-25025
2,740,VOIX,ENTRANT,86b09e4ed3f30d94853e413451f1a69b,20210720,103649,39.0,608-01-09028-02017
3,660,VOIX,SORTANT,b58b587be09e5c1963bf4deba19597cc,20210720,213222,2.0,608-01-09000-36753
4,411,VOIX,ENTRANT,907d1610586c9337c2b19b00eca3bc79,20210720,230420,287.0,608-01-09000-03947


In [73]:
df['international'] = 'domestic'
df['recipient_id'] = ''
df['timestamp'] = df['call_date'].astype(str) + df['call_time'].astype(str)

In [74]:
df['txn_type'].replace({'VOIX':'call', 'SMS':'text'}, inplace=True)

/var/folders/vj/yn7fm0mn7611q4mgvxm7jybm0000gn/T/ipykernel_79784/2394563536.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['txn_type'].replace({'VOIX':'call', 'SMS':'text'}, inplace=True)


In [75]:
df.dropna(subset=["txn_type",'international'], inplace=True)

In [76]:
df.drop(df.columns[0:1], axis=1, inplace=True)

In [77]:
df.to_csv('../data/voix_sms/voix_sms_onnet_cider.csv', sep=",")

In [78]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName('voix_sms').getOrCreate()

In [79]:
df_import = spark.read.csv('../data/voix_sms/voix_sms_onnet_cider.csv', header=True)

In [80]:
df_import.filter(df_import.international.isNull()).head()

24/05/04 12:42:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , txn_type, traffic_direction, caller_id, call_date, call_time, duration, caller_antenna, international, recipient_id, timestamp
 Schema: _c0, txn_type, traffic_direction, caller_id, call_date, call_time, duration, caller_antenna, international, recipient_id, timestamp
Expected: _c0 but found: 
CSV file: file:///Users/cyruspellet/Downloads/features/data/voix_sms/voix_sms_onnet_cider.csv
